In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Pending

In [ ]:
# from scipy.io import loadmat

# link = r"D:\external_data\Experiment4"
# filename = r"\Dataset_PWR_WiFi.mat"
# directory = link + filename

# mat = loadmat(directory)

In [ ]:
# # import experimental dataset 2
# folderpath2 = "D:/external_data/Experiment3/csv_files/exp_2"  # CHANGE THIS IF THE PATH CHANGED
# df_exp2 = import_clean_data('exp2',folderpath2)

In [ ]:
# def evalaute(model, test_loader):
    
#     with torch.no_grad():
        
#         for X_test, y_test in test_loader:
            
#             y_val = model(X_test)
#             predicted = torch.max(y_val,1)[1] 
    
#     arr = confusion_matrix(y_test.view(-1), predicted.view(-1))
#     return arr

# arr = evalaute(model, test_loader)

# lab

In [ ]:
from data.import_data import import_experimental_data
from data.process_data import DatasetObject
from models.train import *

In [ ]:
folderpath1 = "D:/external_data/Experiment3/csv_files/exp_1"  # CHANGE THIS IF THE PATH CHANGED
df_exp1 = import_experimental_data(folderpath1) # import_clean_data('exp1',
df_exp1.head()

In [ ]:
# process data
X_ls, y_ls = seperate_dataframes(df_exp1)
del df_exp1
# DatasetObject
exp_1 = create_datasetobject(X_ls, y_ls)
del X_ls, y_ls

In [ ]:
# extract data
X,y,_ = exp_1.__getitem__([0])

In [ ]:
FIG_M = (18,10)
FIG_S = (10,3)

### visualisation

In [ ]:
section = 0
u = 'user2'

img = df_exp1[(df_exp1['user']==u)].iloc[section*6999:(section+1)*6999,:]
print(img['label'].unique())
img = img.iloc[:,0:90].values
print(img.shape)

In [ ]:
# Graph
plt.figure(figsize=FIG_S)
plt.plot(img[:,::30])
plt.show()
plt.figure(figsize=FIG_S)
plt.plot(img[:,:3])
plt.show()

In [ ]:
line = img[:,30]
n = len(line)
f_hat = np.fft.fft(line,n)
f_hat
psd = f_hat*np.conj(f_hat)/n
psd
freq = (1/n)*np.arange(n)
freq
L = np.arange(1,np.floor(n/2),dtype='int')
L

fig,axs = plt.subplots(2,1)

plt.sca(axs[0])
plt.plot(line)

plt.sca(axs[1])
plt.plot(freq[L],psd[L])


plt.show()

In [ ]:
# Visualise spectrogram
img = img.transpose()
img = img[:,::10]
plt.figure(figsize = (15,50))
plt.imshow(img,cmap='jet')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from data.process_data import stacking
import matplotlib.pyplot as plt



def visual_spectrogram(img,title='spectrogram'):
    
    if len(img.shape) == 3:
        
        img.squeeze()
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(img)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
    plt.figure(figsize = (15,50))
    plt.imshow(img,cmap='jet')
    plt.title(title)
    plt.show()
    
    return 

In [ ]:
import torchvision.models as models
        
#visualize weights for alexnet - first conv layer
alexnet = models.alexnet(pretrained=False)

# Store all conv layer in alexnet
k = []
for m in alexnet.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data.shape)
        k.append(m)
    else:  
        continue


In [ ]:
def visual_single_filter(weight):
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(weight)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
#     plt.figure(figsize = (15,10))
    plt.imshow(weight,cmap='jet')
#     plt.show()
    
    return 

def visualise_filters(layer,figsize=(20,15)):
    
    weights = layer.weight.data

    if weights.shape[0] > weights.shape[1]:
        
        weights = weights.transpose(1,0)
        
    rows = weights.shape[0]
    
    cols = weights.shape[1]
    
    axes = []
    
    fig= plt.figure()

    for n in range(rows):

        for c in range(cols):
            
            axes.append(fig.add_subplot(rows, cols, n*cols+c+1))

            single_filter = weights[n,c]
        
            visual_single_filter(single_filter)
            
#     fig.tight_layout()    
    
    plt.show()     
    
    return 

visualise_filters(example)

### CNN-LSTM

### model factory

In [1]:
import time
from time import gmtime, strftime
# a = time.time()
# model  = train(model, train_loader, criterion, optimizer, 48)
# print(time.time() - a)

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

In [3]:
from models.CNN import Lambda

In [ ]:
class CNN_LSTM_OneToOne(nn.Module):
    def __init__(self,seq_size):
        """
        CNN-LSTM model

        attr:
        seq_size: length of the sequence
        feature_size: feature size of each interval in the sequence

        """
        super(CNN_LSTM_OneToOne, self).__init__()
        self.split = Split_Channel(unsqueeze=False)
        self.cnn = CNN_MultiStream()
        self.lstm = LSTM_Baseline(seq_size=seq_size, feature_size=3*128)

    def forward(self,X):
        X = self.split(X)
        X = [self.cnn(X[i]).unsqueeze(1) for i in range(len(X))]
        X = torch.cat(X,1)
        X = self.lstm(X)
        return X
    
class Split_Channel(nn.Module):
    def __init__(self,unsqueeze=True):
        super(Split_Channel, self).__init__()
        self.unsqueeze = unsqueeze

    def forward(self, x):
        return self.split_channel(x)

    def split_channel(self, x):
        if self.unsqueeze == True:
            return [x[:,i].unsqueeze(1) for i in range(x.shape[1])]
        else:
            return [x[:,i] for i in range(x.shape[1])]


class CNN_SingleStream(nn.Module):

    def __init__(self):
        super(CNN_SingleStream, self).__init__()
        ### 1st ###
        self.conv1 = nn.Conv2d(1,32,kernel_size=3,stride=2)
        self.actv1 = nn.ReLU()
        self.norm1 = Lambda(lambda x:x)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2),stride=(1,1))
        ### 2nd ###
        self.conv2 = nn.Conv2d(32,64,kernel_size=3,stride=2)
        self.actv2 = nn.ReLU()
        self.norm2 = Lambda(lambda x:x)
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2),stride=(1,1))
        ### 3rd ###
        self.conv3 = nn.Conv2d(64,128,kernel_size=3,stride=2)
        self.actv3 = nn.ReLU()
        self.norm3 = Lambda(lambda x:x)
        self.pool3 = Lambda(lambda x:x)
        ### Global_pooling ###
        self.final = nn.AdaptiveAvgPool2d((1,1))

    def forward(self,X):
        X = self.pool1(self.norm1(self.actv1(self.conv1(X))))
        X = self.pool2(self.norm2(self.actv2(self.conv2(X))))
        X = self.pool3(self.norm3(self.actv3(self.conv3(X))))
        X = self.final(X)
        X = torch.flatten(X,1)
        return X

class CNN_MultiStream(nn.Module):

    def __init__(self):
        super(CNN_MultiStream, self).__init__()
        self.split = Split_Channel()
        self.convnet_c1 = CNN_SingleStream()
        self.convnet_c2 = CNN_SingleStream()
        self.convnet_c3 = CNN_SingleStream()
        self.concat = Lambda(lambda a,b,c: torch.cat((a,b,c),-1))

    def forward(self,x):
        a,b,c = self.split(x)
        a = self.convnet_c1(a)
        b = self.convnet_c2(b)
        c = self.convnet_c2(c)
        x = torch.cat((a,b,c),-1)
        return x

In [ ]:
assert False

# Test

In [8]:
np.random.seed(1024)
torch.manual_seed(1024)
torch.set_deterministic(True)

In [9]:
from data.import_data import import_experimental_data
from models.train import * 
from models.CNN import CNN_module, Encoder, Classifier
# from models.self_supervised import *

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [198]:
def evaluate(model, test_loader):
    with torch.no_grad():
        for X_test, y_test in test_loader:
            y_val = model(X_test)
            predicted = torch.max(y_val, 1)[1]
    print(classification_report(y_test.view(-1), predicted.view(-1)))
    arr = confusion_matrix(y_test.view(-1), predicted.view(-1))
    return arr

def make_directory(name, epoch=None, filepath='./models/saved_models/'):
    time = strftime("%Y_%m_%d_%H_%M", gmtime())
    directory = filepath + name + '_checkpoint_' + str(epoch) + '__' + time
    return directory


In [199]:
def train(model, train_loader, criterion, optimizer, end, start = 1, evaluation = None, auto_save = None, **kwargs):
    print('Start Training')
    i = start
    while i <= end:
        print(f"Epoch {i}: ", end='')
        for b, (X_train, y_train) in enumerate(train_loader):
            print(f">", end='')
            optimizer.zero_grad()
            y_pred = model(X_train)
            loss   = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()
        print(f' loss: {loss.tolist()}')
        if i%100 == 0:
            if evaluation == True: 
                array = evaluate(model,test_loader=kwargs['test_loader'])
                print('Evaluation:')
                print(array)
            if auto_save == True:
                directory = make_directory(name=kwargs['name'],epoch=i)
                save_checkpoint(model,optimizer,i,directory)
        i += 1
    return model



In [10]:
# 1. Import data
folderpath1 = "E:/external_data/Experiment3/csv_files/exp_1"
df_exp1 = import_experimental_data(folderpath1)

# 2. Process data
X_ls, y_ls = seperate_dataframes(df_exp1)
del df_exp1

# 3. DatasetObject
exp_1 = create_datasetobject(X_ls, y_ls)
del X_ls,y_ls

# 4. Load data into dataloaders
idxs = [0]
(X_train, y_train,_),(X_test, y_test,_) = exp_1(idxs,return_train_sets=True)
train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test)
del X_train, y_train, X_test, y_test

Found 80 files.
input_user10_bendfwd.csv annotation_user10_bendfwd.csv user10
input_user10_kneel.csv annotation_user10_kneel.csv user10
input_user10_lie.csv annotation_user10_lie.csv user10
input_user10_sit.csv annotation_user10_sit.csv user10
input_user10_sitrotate.csv annotation_user10_sitrotate.csv user10
input_user10_stand.csv annotation_user10_stand.csv user10
input_user10_standrotate.csv annotation_user10_standrotate.csv user10
input_user10_walking.csv annotation_user10_walking.csv user10
input_user1_bendfwd.csv annotation_user1_bendfwd.csv user1
input_user1_kneel.csv annotation_user1_kneel.csv user1
input_user1_lie.csv annotation_user1_lie.csv user1
input_user1_sit.csv annotation_user1_sit.csv user1
input_user1_sitrotate.csv annotation_user1_sitrotate.csv user1
input_user1_stand.csv annotation_user1_stand.csv user1
input_user1_standrotate.csv annotation_user1_standrotate.csv user1
input_user1_walking.csv annotation_user1_walking.csv user1
input_user2_bendfwd.csv annotation_user2

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=list(model.parameters()),lr=1e-4)

In [28]:
epochs = 200
model  = train(model=model, train_loader=train_loader, criterion=criterion, 
                 optimizer=optimizer, end=epochs, start=101, evaluation=True, auto_save=True,
                 name='CnnLstm_v2',test_loader=test_loader)

Start Training
Epoch 101: >>>>>>>>>>>>> loss: 0.1487976461648941
Epoch 102: >>>>>>>>>>>>> loss: 0.1480502188205719
Epoch 103: >>>>>>>>>>>>> loss: 0.1482209861278534
Epoch 104: >>>>>>>>>>>>> loss: 0.1604386568069458
Epoch 105: >>>>>>>>>>>>> loss: 0.1296694427728653
Epoch 106: >>>>>>>>>>>>> loss: 0.13714203238487244
Epoch 107: >>>>>>>>>>>>> loss: 0.12516359984874725
Epoch 108: >>>>>>>>>>>>> loss: 0.12973427772521973
Epoch 109: >>>>>>>>>>>>> loss: 0.12401897460222244
Epoch 110: >>>>>>>>>>>>> loss: 0.1261138767004013
Epoch 111: >>>>>>>>>>>>> loss: 0.1240839883685112
Epoch 112: >>>>>>>>>>>>> loss: 0.13555896282196045
Epoch 113: >>>>>>>>>>>>> loss: 0.11251303553581238
Epoch 114: >>>>>>>>>>>>> loss: 0.11354794353246689
Epoch 115: >>>>>>>>>>>>> loss: 0.10430668294429779
Epoch 116: >>>>>>>>>>>>> loss: 0.11245805025100708
Epoch 117: >>>>>>>>>>>>> loss: 0.12060561031103134
Epoch 118: >>>>>>>>>>>>> loss: 0.10425747185945511
Epoch 119: >>>>>>>>>>>>> loss: 0.13924752175807953
Epoch 120: >>>>>>>>>>>>

In [228]:

# 7. Train
epochs = 200
model  = train__(model=cnn, train_loader=train_loader, criterion=criterion, 
                 optimizer=optimizer, epochs=epochs, start=0, evaluation=True, auto_save=True,
                 name='Baseline',test_loader=test_loader)


Start Training
Epoch 1: >>>>>>>>>>>>> loss: 3.0878446102142334
Epoch 2: >>>>>>>>>>>>> loss: 2.752030611038208
Epoch 3: >>>>>>>>>>>>> loss: 3.13301682472229
Epoch 4: >>>>>>>>>>>>> loss: 3.130406379699707
Epoch 5: >>>>>>>>>>>>> loss: 3.243772029876709
Epoch 6: >>>>>>>>>>>>> loss: 3.0399789810180664
Epoch 7: >>>>>>>>>>>>> loss: 3.3295552730560303
Epoch 8: >>>>>>>>>>>>> loss: 2.9830477237701416
Epoch 9: >>>>>>>>>>>>> loss: 3.184230327606201
Epoch 10: >>>>>>>>>>>>> loss: 3.1230194568634033
Epoch 11: >>>>>>>>>>>>> loss: 3.1196846961975098
Epoch 12: >>>>>>>>>>>>> loss: 3.275430679321289
Epoch 13: >>>>>>>>>>>>> loss: 2.906364679336548
Epoch 14: >>>>>>>>>>>>> loss: 2.8974521160125732
Epoch 15: >>>>>>>>>>>>> loss: 2.918517827987671
Epoch 16: >>>>>>>>>>>>> loss: 2.857963800430298
Epoch 17: >>>>>>>>>>>>> loss: 2.991358995437622
Epoch 18: >>>>>>>>>>>>> loss: 3.1054749488830566
Epoch 19: >>>>>>>>>>>>> loss: 2.9762425422668457
Epoch 20: >>>>>>>>>>>>> loss: 3.0132040977478027
Epoch 21: >>>>>>>>>>>>> l